## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-08-56-55 +0000,nyt,"In Xi’s China, Top General’s Fall Shows Precar...",https://www.nytimes.com/2026/01/26/world/asia/...
1,2026-01-26-08-41-00 +0000,wsj,"Gold Surges Past $5,000 as Canada Tariff Risk,...",https://www.wsj.com/finance/commodities-future...
2,2026-01-26-08-40-00 +0000,wsj,Eurozone Government Bond Yields Fall as Supply...,https://www.wsj.com/finance/jgbs-rise-as-yens-...
3,2026-01-26-08-39-00 +0000,wsj,Further Downside for U.S. Dollar Should Be Lim...,https://www.wsj.com/finance/currencies/yen-str...
4,2026-01-26-08-37-47 +0000,nyt,At Least 18 Dead After Ferry Carrying 350 Sink...,https://www.nytimes.com/2026/01/25/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2427/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,32
48,shooting,28
109,minneapolis,22
59,ice,14
60,storm,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,134
97,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...,93
197,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,92
104,2026-01-25-22-29-48 +0000,bbc,NRA and other US gun activists push back at Tr...,https://www.bbc.com/news/articles/cnvg812n01no...,90
6,2026-01-26-08-29-04 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,134,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
45,52,2026-01-26-04-30-19 +0000,nypost,Violent anti-ICE protesters swarm Minnesota ho...,https://nypost.com/2026/01/25/us-news/anti-ice...
80,51,2026-01-26-00-14-51 +0000,nypost,"Winter Storm Fern rages across US, canceling n...",https://nypost.com/2026/01/25/us-news/winter-s...
97,37,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
106,30,2026-01-25-22-22-19 +0000,nypost,China’s top general accused of leaking country...,https://nypost.com/2026/01/25/world-news/china...
188,27,2026-01-25-11-00-39 +0000,nypost,NYC homeless living high life under Mayor Mamd...,https://nypost.com/2026/01/25/us-news/nyc-home...
11,25,2026-01-26-07-47-57 +0000,nypost,Missing Kansas teacher found dead in snow afte...,https://nypost.com/2026/01/26/us-news/missing-...
57,25,2026-01-26-02-09-44 +0000,startribune,The latest: Mounting calls for DHS surge to en...,https://www.startribune.com/ice-raids-minnesot...
94,23,2026-01-25-23-09-32 +0000,wapo,ICU nurse fatally shot by Border Patrol in Min...,https://www.washingtonpost.com/nation/2026/01/...
67,21,2026-01-26-01-21-56 +0000,nypost,Daughter of top Iranian official ousted from E...,https://nypost.com/2026/01/25/us-news/emory-un...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
